In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd './drive/MyDrive/hasoc/task3'

/content/drive/MyDrive/hasoc/task3


In [ ]:
!pip install datasets
!pip install transformers
!pip install accelerate -U
!pip install evaluate
!pip install seqeval

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import random
import torch
import numpy as np
from datasets import Dataset
import pandas as pd

GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [ ]:
df = pd.read_csv('train.csv', sep = '|', index_col = 0)

# split sentences to words and split tags
df['tokens'] = df['sentence'].apply(lambda x: x.split())
df['bio_split'] = df['bio'].apply(lambda x: x.split())

# chech whether length of tokens and length of tags are equal for all rows of dataframe
for idx, row in df.iterrows():
    assert(len(row[3]) == len(row[4]))

In [ ]:
label_list = set()
for idx, row in df.iterrows():
    label_list = label_list.union(set(row[4]))

label_list = list(label_list)

num_labels = len(label_list)
id2label = {id: label for id, label in enumerate(label_list)}
label2id = {label: id for id, label in enumerate(label_list)}

df['tags'] = df['bio_split'].apply(lambda tags_list: [label2id[tag] for tag in tags_list])


In [ ]:
df

,sentence,span,bio,tokens,bio_split,tags
Id,,,,,,
0,"Say it loud , say it clear , illegal #immigran...","{'start': [8, 9], 'end': [8, 9]}",O O O O O O O O B I O O O O O,"[Say, it, loud, ,, say, it, clear, ,, illegal,...","[O, O, O, O, O, O, O, O, B, I, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0]"
1,"Islam is not a religion, its just an excuse fo...","{'start': [0, 10], 'end': [4, 13]}",B I I I I O O O O O B I I I O O O O,"[Islam, is, not, a, religion,, its, just, an, ...","[B, I, I, I, I, O, O, O, O, O, B, I, I, I, O, ...","[2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, ..."
2,@user Care About Illegals Breaking U.S #Immigr...,"{'start': [40], 'end': [40]}",O O O O O O O O O O O O O O O O O O O O O O O ...,"[@user, Care, About, Illegals, Breaking, U.S, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Behold the future DACA America - unless we Man...,"{'start': [10, 17], 'end': [14, 18]}",O O O O O O O O O O B I I I I O O B I O O O O ...,"[Behold, the, future, DACA, America, -, unless...","[O, O, O, O, O, O, O, O, O, O, B, I, I, I, I, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, ..."
4,No that little tattoo of a flower on your ankl...,"{'start': [14, 20, 25, 26], 'end': [14, 21, 25...",O O O O O O O O O O O O O O B O O O O O B I O ...,"[No, that, little, tattoo, of, a, flower, on, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
...,...,...,...,...,...,...
2416,but if you are a nigger you can beat rape your...,"{'start': [5, 8], 'end': [5, 9]}",O O O O O B O O B I O O O O O O O O O O O O O ...,"[but, if, you, are, a, nigger, you, can, beat,...","[O, O, O, O, O, B, O, O, B, I, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, ..."
2417,Immigrants Come LEGALLY like 1.5 Milion Do Eve...,"{'start': [0, 2, 13, 20], 'end': [0, 2, 13, 20]}",B O B O O O O O O O O O O B O O O O O O B O O ...,"[Immigrants, Come, LEGALLY, like, 1.5, Milion,...","[B, O, B, O, O, O, O, O, O, O, O, O, O, B, O, ...","[2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
2418,lol kike,"{'start': [1], 'end': [1]}",O B,"[lol, kike]","[O, B]","[0, 2]"


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
example = df.iloc[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'say',
 'it',
 'loud',
 ',',
 'say',
 'it',
 'clear',
 ',',
 'illegal',
 '#',
 'immigrants',
 'are',
 'not',
 'welcome',
 'here',
 '@',
 'user',
 '[SEP]']

In [ ]:
ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.1)
ds['validation'] = ds.pop('test')
ds.num_rows

{'train': 2178, 'validation': 243}

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = ds.map(tokenize_and_align_labels, batched=True)
# tokenized_dataset = ds.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/243 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./token-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.343561,0.403651,0.487745,0.441731,0.881977
2,No log,0.318429,0.471239,0.522059,0.495349,0.890622


TrainOutput(global_step=274, training_loss=0.3686862305133012, metrics={'train_runtime': 40.4671, 'train_samples_per_second': 107.643, 'train_steps_per_second': 6.771, 'total_flos': 70619787800100.0, 'train_loss': 0.3686862305133012, 'epoch': 2.0})

In [ ]:
# trainer.push_to_hub()

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("mohammadmostafameb/token-classifier")
text = "says the kike cunt trying to charge for premium content how that working for you obviously nobody gives a fuck what you have to say cuz you are a fucking joke go blow some wetbacks behind the burrito stand and you might make a buck lick balls heebhole premium cunt contributor"
# inputs = tokenizer(text, return_tensors="pt").to('cuda')
inputs = tokenizer(text, return_tensors="pt").to('cpu')

In [ ]:
# from transformers import AutoModelForTokenClassification

# model = AutoModelForTokenClassification.from_pretrained("mohammadmostafameb/token-classifier")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

In [ ]:
print(len(predicted_token_class))
print(len(text.split()))

61
51


# new generation

In [ ]:
df_test = pd.read_csv('test.csv', sep = '|', index_col = 0)
df_test['tokens'] = df_test['sentence'].apply(lambda x: x.split())
# df_test
tds = Dataset.from_pandas(df_test)


def test_tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    res = []
    for i, token in enumerate(examples[f"tokens"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        token_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                token_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                # token_ids.append(label[word_idx])
                token_ids.append(word_idx)
            else:
                token_ids.append(-100)
            previous_word_idx = word_idx
        res.append(token_ids)

    tokenized_inputs['token_ids'] = res
    return tokenized_inputs

In [ ]:
tds

Dataset({
    features: ['sentence', 'tokens', 'Id'],
    num_rows: 606
})

In [ ]:
df_test = pd.read_csv('test.csv', sep = '|', index_col = 0)
df_test['tokens'] = df_test['sentence'].apply(lambda x: x.split())

tds = Dataset.from_pandas(df_test)

def test_tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    res = []
    for i, token in enumerate(examples[f"tokens"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        token_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                token_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                # token_ids.append(label[word_idx])
                token_ids.append(word_idx)
            else:
                token_ids.append(-100)
            previous_word_idx = word_idx
        res.append(token_ids)

    tokenized_inputs['token_ids'] = res
    return tokenized_inputs


tokenized_test_dataset = tds.map(test_tokenize_and_align_labels, batched=True)

# tokenizer.decode(tokenized_test_dataset[0]['input_ids'])

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

In [ ]:
# Evaluation
all_predictions = []
fucked_up = []

model.eval()
for test_data in tokenized_test_dataset:
    input_ids = torch.tensor([test_data['input_ids']], dtype=torch.long).to('cuda')
    attention_mask = torch.tensor([test_data['attention_mask']], dtype=torch.long).to('cuda')
    token_ids = test_data['token_ids']
    Id = test_data['Id']
    print(f"processing {Id} ...")

    with torch.no_grad():
        outputs = model(input_ids = input_ids, attention_mask = attention_mask)

    predictions = outputs.logits.argmax(dim=-1)

    true_predictions = []

    for idx, prediction in enumerate(predictions[0]):
        if token_ids[idx] != -100:
            true_predictions.append(id2label[prediction.item()])
    try:
        assert(len(true_predictions)==len(df_test.iloc[Id]['tokens']))
    except:
        print(f"predictions: {len(true_predictions)}, sentence: {len(df_test.iloc[Id]['sentence'].split())}")
        fucked_up.append({'predictions':true_predictions, 'sentence': df_test.iloc[Id]['sentence'].split()})
    all_predictions.append(true_predictions)


    # print(len(tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)), tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True))
    # break

processing 0 ...
processing 1 ...
processing 2 ...
processing 3 ...
processing 4 ...
processing 5 ...
processing 6 ...
processing 7 ...
processing 8 ...
processing 9 ...
processing 10 ...
processing 11 ...
processing 12 ...
processing 13 ...
processing 14 ...
processing 15 ...
processing 16 ...
processing 17 ...
processing 18 ...
processing 19 ...
processing 20 ...
processing 21 ...
processing 22 ...
processing 23 ...
processing 24 ...
processing 25 ...
processing 26 ...
processing 27 ...
processing 28 ...
processing 29 ...
processing 30 ...
processing 31 ...
processing 32 ...
processing 33 ...
processing 34 ...
processing 35 ...
processing 36 ...
processing 37 ...
processing 38 ...
processing 39 ...
processing 40 ...
processing 41 ...
processing 42 ...
processing 43 ...
processing 44 ...
processing 45 ...
processing 46 ...
processing 47 ...
processing 48 ...
processing 49 ...
processing 50 ...
processing 51 ...
processing 52 ...
processing 53 ...
processing 54 ...
processing 55 ...
pr

In [ ]:
df_test['labels'] = all_predictions
df_test['bio'] = df_test['labels'].apply(lambda x: ' '.join(x))

In [ ]:
df_test.to_csv('results.csv')

In [ ]:
all_predictions
true_predictions
print(df_test.iloc[0]['sentence'])
print(len(df_test.iloc[0]['sentence'].split()))


In [ ]:
all_predictions[284]

In [ ]:
df_test.iloc[284]['sentence'].split()

In [ ]:
print(tokenized_test_dataset[284]['input_ids'])
print(tokenizer.decode(tokenized_test_dataset[284]['input_ids'], clean_up_tokenization_spaces=False))
print(tokenized_test_dataset[284]['tokens'])
print(tokenized_test_dataset[284]['token_ids'])

[101, 1048, 2213, 7011, 2080, 1996, 2128, 7559, 5732, 2791, 10930, 2941, 21690, 1037, 8239, 2613, 11382, 3489, 2003, 2025, 2474, 14536, 2075, 2017, 14163, 29519, 2272, 2006, 1030, 5310, 2017, 2024, 2025, 6016, 2115, 2190, 2000, 20716, 27082, 2149, 102]
[CLS] lmfao the retardedness yo actually stabbing a fucking real kike is not larping you muppet come on @ user you are not sending your best to bullycide us [SEP]
['lmfao', 'the', 'retardedness', 'yo', 'actually', 'stabbing', 'a', 'fucking', 'real', 'kike', 'is', 'not', 'larping', 'you', 'muppet', 'come', 'on', '@user', '\u200d', 'you', 'are', 'not', 'sending', 'your', 'best', 'to', 'bullycide', 'us']
[-100, 0, -100, -100, -100, 1, 2, -100, -100, -100, 3, 4, 5, 6, 7, 8, 9, -100, 10, 11, 12, -100, -100, 13, 14, -100, 15, 16, 17, -100, 19, 20, 21, 22, 23, 24, 25, 26, -100, 27, -100]


In [ ]:
print(len(tokenized_test_dataset[284]['input_ids']))
print(len(tokenizer.decode(tokenized_test_dataset[284]['input_ids'], clean_up_tokenization_spaces=False).split()))
print(len(tokenized_test_dataset[284]['tokens']))
print(len(tokenized_test_dataset[284]['token_ids']))
print(max(tokenized_test_dataset[284]['token_ids']))


41
30
28
41
27


In [ ]:
# fucked up
# 284
# 423
# 434

id = 423
true_predictions = all_predictions[id]

print(f"{len(true_predictions)} != {len(df_test.iloc[id]['tokens'])}")
print(list(zip(df_test.iloc[id]['tokens'],true_predictions)))
print(df_test.iloc[id]['tokens'][-1])

22 != 24
[('@user', 'B'), ('\u200d', 'O'), ('the', 'I'), ('jews', 'O'), ('are', 'O'), ('controlling', 'O'), ('my', 'O'), ('weather', 'B'), ('again', 'O'), ('i', 'I'), ('am', 'I'), ('a', 'I'), ('get', 'O'), ('that', 'B'), ('kike', 'O'), ('alf', 'B'), ('for', 'O'), ('this', 'O'), ('shit', 'O'), ('you', 'B'), ('with', 'B'), ('me', 'B')]
‍


In [ ]:
# tokenized_test_dataset[id]['tokens']
tokenized_test_dataset[id]['sentence']

'lmfao the retardedness yo actually stabbing a fucking real kike is not larping you muppet come on @user \u200d you are not sending your best to bullycide us'

In [ ]:
len(tokenized_test_dataset[id]['tokens'])

28

In [ ]:
# test_tokenize_and_align_labels(tokenized_test_dataset[id])
examples = tokenized_test_dataset[id]
tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

In [ ]:
len(examples["tokens"])

28

In [ ]:
# len(tokenized_inputs['input_ids'])
# print(tokenized_inputs)
for k, v in tokenized_inputs.items():
    print(f'{k}: {v}')

In [ ]:
len(tokenized_inputs.word_ids(batch_index = 0)), len(tokenized_test_dataset[id]['tokens'])

(41, 28)

In [ ]:
tokenized_inputs.word_ids(batch_index = 0)

In [ ]:
list(zip())

In [ ]:
def test_tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    res = []
    for i, token in enumerate(examples[f"tokens"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        token_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                token_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                # token_ids.append(label[word_idx])
                token_ids.append(word_idx)
            else:
                token_ids.append(-100)
            previous_word_idx = word_idx
        res.append(token_ids)

    tokenized_inputs['token_ids'] = res
    return tokenized_inputs